### Advanced Data Analysis
Jim Chung
<br>
<br>
I implemented the idea of Michael Utomo that the random forest model can identify the strongest predictor. <br><br> X: 8 selected features <br> y: Grade -  can be changed according to our interests

In [154]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn.preprocessing import OneHotEncoder, Imputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import f1_score, make_scorer

import matplotlib.pyplot as plt
import seaborn

%matplotlib inline

In [155]:
# Data Loaded
data = pd.read_csv("loan.csv")

In [156]:
# See the info
data.describe()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,annual_inc,dti,delinq_2yrs,...,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
count,8.873790e+05,8.873790e+05,887379.000000,887379.000000,887379.000000,887379.000000,887379.000000,8.873750e+05,887379.000000,887350.000000,...,21372.000000,18617.000000,21372.000000,21372.000000,21372.000000,21372.000000,8.171030e+05,21372.000000,21372.000000,21372.000000
mean,3.246513e+07,3.500182e+07,14755.264605,14741.877625,14702.464383,13.246740,436.717127,7.502759e+04,18.157039,0.314442,...,36552.811389,71.535317,1.389060,2.975482,5887.979740,60.831939,3.206862e+04,0.943945,1.537058,1.977307
std,2.282734e+07,2.411335e+07,8435.455601,8429.897657,8442.106732,4.381867,244.186593,6.469830e+04,17.190626,0.862244,...,43103.833619,23.022862,1.520129,2.631886,5284.701239,20.013254,3.749826e+04,1.446872,2.723339,2.874067
min,5.473400e+04,7.047300e+04,500.000000,500.000000,0.000000,5.320000,15.670000,0.000000e+00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,-4.000000
25%,9.206643e+06,1.087713e+07,8000.000000,8000.000000,8000.000000,9.990000,260.705000,4.500000e+04,11.910000,0.000000,...,10251.500000,58.600000,0.000000,1.000000,2411.000000,47.700000,1.390000e+04,0.000000,0.000000,0.000000
50%,3.443327e+07,3.709528e+07,13000.000000,13000.000000,13000.000000,12.990000,382.550000,6.500000e+04,17.650000,0.000000,...,24684.500000,74.900000,1.000000,2.000000,4483.000000,61.900000,2.370000e+04,0.000000,0.000000,2.000000
75%,5.490814e+07,5.847135e+07,20000.000000,20000.000000,20000.000000,16.200000,572.600000,9.000000e+04,23.950000,0.000000,...,47858.000000,87.600000,2.000000,4.000000,7771.500000,75.200000,3.980000e+04,1.000000,2.000000,3.000000
max,6.861706e+07,7.354484e+07,35000.000000,35000.000000,35000.000000,28.990000,1445.460000,9.500000e+06,9999.000000,39.000000,...,878459.000000,223.300000,22.000000,43.000000,127305.000000,151.400000,9.999999e+06,17.000000,35.000000,32.000000


In [157]:
# Select the columns to be considered
col = 'loan_amnt,funded_amnt,int_rate,emp_length,home_ownership,annual_inc,last_pymnt_amnt'.split(',')
X = data[col]
y = data.grade

In [158]:
# Encode the targets: grade
oh = OneHotEncoder()
y = oh.fit_transform(y.factorize(sort=True)[0].reshape(-1,1)).toarray()

In [159]:
# Preprocessing
X['emp_length'] = X['emp_length'].str.extract('(\d+)',)
X['home_ownership'] = X['home_ownership'].factorize(sort=True)[0]

# Missing Value
imputer = Imputer(strategy='median')
X = imputer.fit_transform(X)

# Test set split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [164]:
# Grid search to find the optimal parameters
param_grid = [
    {'n_estimators': [3, 10, 30, 100], 'max_features': [2, 4, 6, 8]}
]

# Fit random forests
clf = ensemble.RandomForestClassifier()

# accuracy is not a good metric for multi-class problems -> f1 score
scorer = make_scorer(f1_score, average='weighted')
grid_search = GridSearchCV(clf, param_grid, cv=5, scoring=scorer)

In [ ]:
# Fit the data
grid_search.fit(X_train, y_train)

In [ ]:
# Metrics
cvres = grid_search.cv_results_
for f1_score, params in zip(cvres['f1_score'], cvres['params']):
    print(f1_score, params)

In [ ]:
# Feature importances
grid_search.best_estimator_.feature_importances_

In [ ]:
y_pred = grid_serach.best_estimator_.predict(X_train)
print(f1_score(y_test, y_pred))